In [ ]:
from transformers import Trainer, TrainingArguments, DistilBertForSequenceClassification, DistilBertTokenizer
from datasets import load_dataset
import torch
from torch.quantization import quantize_dynamic

# 데이터셋 불러오기
dataset = load_dataset("imdb")
train_dataset = dataset["train"]
val_dataset = dataset["test"]

In [ ]:
# DistilBERT 모델과 토크나이저 불러오기
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
# 모델 양자화 (8-bit 정수 양자화)
model = quantize_dynamic(
    model, 
    {torch.nn.Linear},  # 양자화할 레이어 지정 (여기서는 Linear 레이어)
    dtype=torch.qint8    # 8-bit 양자화
)

In [ ]:
# 데이터셋 토큰화
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

In [ ]:
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

In [ ]:
# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir="./results",                # 훈련 결과 저장 디렉토리
    num_train_epochs=3,                    # 훈련 epoch 수
    per_device_train_batch_size=8,         # 배치 크기
    per_device_eval_batch_size=8,          # 평가 배치 크기
    logging_dir="./logs",                  # 로그 파일 저장 디렉토리
    logging_steps=500,                     # 로그를 저장할 주기
    evaluation_strategy="epoch",           # 에포크마다 평가
    save_steps=500,                        # 500번째 배치마다 모델 저장
    load_best_model_at_end=True,           # 최적 모델을 마지막에 로드
    metric_for_best_model="accuracy",     # 모델의 최적 성능 기준 (정확도)
)

In [ ]:
# Trainer 설정
trainer = Trainer(
    model=model,                           # 훈련할 모델
    args=training_args,                    # 훈련 인자
    train_dataset=train_dataset,           # 훈련 데이터셋
    eval_dataset=val_dataset,              # 평가 데이터셋
)

In [ ]:
# 훈련 시작
trainer.train()

#### 양자화(Quantization) 적용 시 모델의 변화
- **양자화(Quantization)**는 모델의 파라미터와 계산을 더 적은 비트로 표현하여 모델 크기를 줄이고 계산 효율성을 높이는 기술입니다. 양자화는 특히 딥러닝 모델의 추론(예측) 단계에서 유용하며, 모델 경량화 및 속도 향상을 목표로 합니다.
- 양자화는 보통 *32-bit floating point(FP32)* 형식으로 훈련된 모델을 *8-bit integer(INT8)* 등으로 변환하는 방식입니다. 양자화된 모델은 **메모리 사용량을 절감** 하고, **연산 속도를 향상** 시킬 수 있습니다. 그러나, 양자화가 모델의 성능에 미치는 영향도 고려해야 합니다.

#### 양자화의 종류
1. Weights Quantization (가중치 양자화): 모델의 가중치를 8-bit 정수로 변환하여 저장합니다.
2. Activations Quantization (활성화 양자화): 모델의 활성화 값도 양자화하여 처리합니다.
3. Full-Precision Inference (전체 정밀도 추론): 훈련 시 사용하는 정밀도를 그대로 사용하여 추론을 진행합니다.
4. Dynamic Quantization: 양자화 시점에서 동적으로 적용하는 방식으로, 훈련 후 즉시 모델을 양자화하여 적용할 수 있습니다.
5. Static Quantization: 사전에 양자화를 수행하고, 변환된 모델을 사용하여 추론을 진행합니다.

#### 양자화를 적용했을 때의 변화

1. 모델 크기 감소:
- 양자화를 통해 모델의 크기는 크게 줄어듭니다. 예를 들어, 32-bit 가중치를 8-bit로 변환하면 모델 크기가 약 4배 작아집니다.
- 이는 특히 모바일 기기나 엣지 디바이스와 같은 제한된 자원을 가진 환경에서 중요한 이점입니다.

2. 연산 속도 향상:
- 8-bit 정수 연산은 32-bit 부동소수점 연산보다 연산 속도가 빠릅니다. 특히 하드웨어 가속기(예: GPU, TPU, 혹은 특수 하드웨어에서 지원되는 INT8 연산)에 최적화되어 있어, 추론 속도가 크게 향상됩니다.
- 양자화된 모델은 하드웨어에 따라 더 빠르게 실행될 수 있습니다.

3. 메모리 사용 절감:
- 양자화는 메모리 사용량을 절감합니다. 8-bit로 변환된 가중치와 활성화는 32-bit로 표현된 값보다 4배 적은 메모리를 차지합니다.
- 이를 통해 메모리 대역폭이 더 적게 사용되고, 캐시 효율성이 향상될 수 있습니다.

4. 성능 감소 가능성:
- 정밀도 손실: 양자화된 모델은 일부 *정밀도 손실* 을 겪을 수 있습니다. 32-bit 실수 값 대신 8-bit 정수로 계산하므로 일부 미세한 차이가 생길 수 있습니다.
- 성능 저하: 특히 고정밀도가 중요한 모델에서는 양자화로 인한 성능 저하가 발생할 수 있습니다. 그러나 일부 경우에는 양자화가 *성능에 큰 영향을 미치지 않거나 미세한 손실* 로 충분히 견딜 수 있습니다.
- 성능 회복을 위한 재학습: 양자화를 적용한 후 성능이 저하되었을 때, **재학습(fine-tuning)**을 통해 성능을 회복할 수 있습니다.

5. 전력 소모 절감:
- 양자화된 모델은 전력 소모를 줄이는 데 유리합니다. 낮은 비트폭으로 연산을 수행하므로, 동일한 연산을 처리할 때 전력 소비가 적습니다.
- 이는 특히 모바일 기기나 엣지 디바이스와 같은 전력 소비가 중요한 환경에서 유리합니다.
